In [1]:
import sys
import os
from pprint import pprint
sys.path.append(os.path.abspath("../../../"))
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, HTML
from IPython.display import Markdown
import ipywidgets as iw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
import usure.common.logging as usurelogging
from usure.config import config
from usure.classification.infrastructure import (
     BasicSentenceCleaner 
    ,FileLabeledCommentsDao
    ,FileWordVectorsRep
    ,CsvLabeledCommentsDao)
from usure.classification.core import SvmLab, ClassifierInput, LabReport, WordVectorsService, LabeledComments
from usure.classification.infrastructure import FileModelDao
import usure.classification.ui.utils as ui

Using TensorFlow backend.


In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = lines => false

<IPython.core.display.Javascript object>

In [4]:
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_colwidth', 200)

In [5]:
cleaner = BasicSentenceCleaner(config.assets)
comments_dao = CsvLabeledCommentsDao(config.sets, cleaner)
wv_rep = FileWordVectorsRep(config.embeddings)
model_dao = FileModelDao(config.models)
lab = SvmLab(model_dao)
comments = comments_dao.get("train.csv")
test_comments = comments_dao.get("test.csv")
test_inputs = []

In [6]:
def classification_lab_report(word_vector_no, input, lab):
    display(Markdown(f'## {word_vector_no}. Creating models with embeddings: {input.embeddings_name}'))
    lab_report = lab.train_by_stratifiedkfold(input, folds=10)
    return lab_report

def model_architecture_summary(lab_report):
    display(Markdown(f'### Mean Accuracies and losses'))
    display(lab_report.summary)
    display(Markdown(f'### Accuracies and losses by Stratified KFold (folds with balanced class distributions)'))
    model_reports_df = ui.model_reports_to_DataFrame(lab_report.model_reports)
    display(model_reports_df) 
    
def show_statistics_by_model(word_vector_no, word_vector_name, lab_report):
    """Shows statistics by models created by every folds"""
    display(Markdown(f'## {word_vector_no+1}. {word_vector_name}'))
    for fold_no, model_report in enumerate(lab_report.model_reports):
        display(Markdown(f'### Fold number: {fold_no+1}. Name: {model_report.name}'))
        ui.plot_learning_curves_if_epoches(model_report)
        display(Markdown(f'### Training'))
        print(model_report.training.classification_report)
        confu_mtx = model_report.training.confusion_matrix
        display(confu_mtx)
        display(Markdown(f'### Validation'))
        print(model_report.validation.classification_report)
        confu_mtx = model_report.validation.confusion_matrix
        display(confu_mtx)
        display(Markdown(f'### Test'))
        test_metrics, labeled_predictions = lab.test(model_report.name, test_inputs[word_vector_no])
        print(f"Accuracy: {test_metrics.accuracy}")
        print(f"Loss: {test_metrics.loss}")
        print(test_metrics.classification_report)
        display(test_metrics.confusion_matrix)
        test_labeled_comments = test_inputs[word_vector_no].labeled_comments
        labeled_comments = LabeledComments(f"{model_report.name}-{word_vector_name}.cnn.csv",test_labeled_comments.comments ,labeled_predictions)
        comments_dao.save_from_origin(labeled_comments, "test.csv")

In [ ]:
#%matplotlib notebook
display(Markdown(f'# Suport Vector Machine'))
word_vectors = wv_rep.get_all()
word_vector_names = []
lab_reports = []

#word_vectors = [word_vectors.__next__()] #comment!!!!!
for i, word_vector in enumerate(word_vectors):
    wv_service = WordVectorsService(word_vector)
    test_inputs.append(ClassifierInput(test_comments, wv_service))
    input = ClassifierInput(comments, wv_service)
    lab_report = classification_lab_report(i+1, input, lab)
    model_architecture_summary(lab_report)
    word_vector_names.append(word_vector.name)
    lab_reports.append(lab_report)
    
for i, word_vector_name in enumerate(word_vector_names): 
    lab_report = lab_reports[i]
    show_statistics_by_model(i, word_vector_name, lab_report)
    

# Suport Vector Machine

## 1. Creating models with embeddings: CorpusFBCR2013.txt.usu.bw.kvs

### Mean Accuracies and losses

,train_acc_mean,train_acc_stdev,val_acc_mean,val_acc_stdev
0,0.603105,0.009009,0.506229,0.033513


### Accuracies and losses by Stratified KFold (folds with balanced class distributions)

...

### Validation

              precision    recall  f1-score   support

           N       0.49      0.78      0.61        46
         NEU       0.00      0.00      0.00        15
        NONE       0.50      0.14      0.21        22
           P       0.59      0.65      0.62        34

   micro avg       0.52      0.52      0.52       117
   macro avg       0.40      0.39      0.36       117
weighted avg       0.46      0.52      0.46       117



,N,NEU,NONE,P
Pred/True,,,,
N,36,11,16,10
NEU,0,0,1,0
NONE,1,0,3,2
P,9,4,2,22


### Test

Accuracy: 0.4892887746358183
Loss: 1.1800753423150212
              precision    recall  f1-score   support

           N       0.47      0.87      0.61       456
         NEU       0.00      0.00      0.00       148
        NONE       0.47      0.09      0.15       224
           P       0.57      0.45      0.50       339

   micro avg       0.49      0.49      0.49      1167
   macro avg       0.38      0.35      0.32      1167
weighted avg       0.44      0.49      0.41      1167

